<a href="https://colab.research.google.com/github/showman-sharma/ALC_tableau/blob/main/ALCtableau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extracting Abox

In [ ]:
def copy(L):
  # print(L)
  L2 = {}
  for a in L:
    try:
      L2[a] = L[a].copy()
    except:
      L2[a] = L[a]  
  return(L2)  

In [ ]:
!pip3 install xmltodict
import xmltodict
import ast
import re

with open("/content/MAN-P7-Input-KB.xml") as xml_file:
     
    kb_dict = xmltodict.parse(xml_file.read())
    xml_file.close()

with open("/content/MAN-P7-Input-Query.xml") as xml_file:
     
    query_dict = xmltodict.parse(xml_file.read())
    xml_file.close()

with open("/content/MAN-P6-KB.xml") as xml_file:
     
    abox_dict = xmltodict.parse(xml_file.read())
    xml_file.close()

In [ ]:
print('EXAMPLE 1')

EXAMPLE 1


In [ ]:
Ind = []
for i in abox_dict['KB']['DifferentIndividuals']['INDIVIDUAL']:
  Ind.append(i)

Rel = {}
for r in abox_dict['KB']['ObjectProperty']:
  Rel[r['ROLE']] = {}

Class = {}
for f in abox_dict['KB']['Individual']:
  if 'Types' in f:
    if f['Types']['CONCEPT'] not in Class:
      Class[f['Types']['CONCEPT']] = []
    Class[f['Types']['CONCEPT']].append(f['INDIVIDUAL'])
  if 'Facts' in f:
    if f['Facts']['FACT']['ROLE'] not in Rel:
      Rel[f['Facts']['FACT']['ROLE']] = {}
    if f['INDIVIDUAL'] not in Rel[f['Facts']['FACT']['ROLE']]:
      Rel[f['Facts']['FACT']['ROLE']][f['INDIVIDUAL']] = []


    Rel[f['Facts']['FACT']['ROLE']][f['INDIVIDUAL']].append(f['Facts']['FACT']['INDIVIDUAL'])

In [ ]:
print('ABOX:')
print('Classes:',Class)
print('Relations:',Rel)

ABOX:
Classes: {'Mother': ['Priya'], 'Bachelor': ['Dhruv']}
Relations: {'parentOf': {'Priya': ['Dhruv']}, 'husbandOf': {}}


# ABox

In [ ]:
Q = [query_dict['KB']['Individual']['INDIVIDUAL']]

nnf = query_dict['KB']['Individual']['Types']
# print(nnf)
str_nnf = str(nnf)
str_nnf = str_nnf.replace('[', '')
str_nnf = str_nnf.replace(']', '')

str_nnf = str_nnf.replace('{', '[')
str_nnf = str_nnf.replace('}', ']')
str_nnf = str_nnf.replace(':', ',')
str_nnf = str_nnf.replace("'CONCEPT', ", '')
str_nnf = str_nnf.replace("'NOT', [", "['!', ")
# str_nnf = str_nnf.replace("],", ",")
str_nnf = str_nnf.replace("'OR', [", "['|', ")
str_nnf = str_nnf.replace("'AND', [", "['&', ")
str_nnf = str_nnf.replace("'EXISTS', [", "['*', ")
str_nnf = str_nnf.replace("'FORALL', [", "['?', ")
str_nnf = str_nnf.replace("'ROLE', ", '')

# print(str_nnf)
Q.append(ast.literal_eval(str_nnf)[0])

In [ ]:
print('Query:',Q)

Query: ['Dhruv', ['*', 'parentOf', 'Person']]


In [ ]:
def notof(X):
  if isinstance(X,str):
    return ['!',X]
  elif X[0] == '!':
    return X[1]
  else:
    assert False,  "WRONG FORMAT"    

In [ ]:
#Fill L from Abox
def abox2L(Rel,Class):
  L = {};Rr = {}
  for c in Class:
    for a in Class[c]:
      try:
        L[a].append(c)
      except:
        L[a] = [c]
  for R in Rel:
    for a in Rel[R]:
      for b in Rel[R][a]:
        try:
          Rr[a+','+b].append(R)
        except:
          Rr[a+','+b] = [R]
  return L,Rr      


In [ ]:
L0 ,R0= abox2L(Rel,Class)

In [ ]:
print('Before adding not of query:',L0,R0)

Before adding not of query: {'Priya': ['Mother'], 'Dhruv': ['Bachelor']} {'Priya,Dhruv': ['parentOf']}


In [ ]:
def addQuery(Q,La,Ra):
  L = copy(La);R = copy(Ra)
  a = Q[0]
  c = Q[1]
  if isinstance(c,list):
    if c[0]=='*':
      Tnow = ['?',c[1],notof(c[2])]
      rel = Tnow[1]
      cl = Tnow[2]
      L[a].append(Tnow)
      for item in L:
        if a+','+item in R and rel in R[a+','+item] and cl not in L[item]:
          L[item].append(cl)
      return L,R    

    elif c[0]=='?':
      Tnow = ['*',c[1],notof(c[2])]
      rel = Tnow[1]
      cl = Tnow[2]
      Lnew = copy(L)
      Rnew = copy(R)
      for a in L:
        Lnew[a].append(Tnow)
        for item in L:
          if not ((a+','+item) in R and rel in R[a+','+item] and cl in L[item]):
            Lnew['var'+a] = [cl]
            Rnew[a+','+'var'+a] = [rel]
      return Lnew,Rnew      
    elif c[0]=='!':
      L[a].append(c[1])
      return L,R

    else:
      assert False; "WRONG FORMAT" 
  else:
    L[a].append(notof(c))   
    return L,R      

In [ ]:
L1,R1 = addQuery(Q,L0,R0)

In [ ]:
print('After adding not of query:',L1,R1)

After adding not of query: {'Priya': ['Mother'], 'Dhruv': ['Bachelor', ['?', 'parentOf', ['!', 'Person']]]} {'Priya,Dhruv': ['parentOf']}


#Tbox encoding

* ! = Not
* | = Or
* & = And
* \* = there exists/ some in
* ? = for all/ only in


In [ ]:
Tbox = []
for s in kb_dict['KB']['Class']:
  nnf = s['EquivalentTo']
  # print(nnf)
  str_nnf = str(nnf)
  str_nnf = str_nnf.replace('[', '')
  str_nnf = str_nnf.replace(']', '')

  str_nnf = str_nnf.replace('{', '[')
  str_nnf = str_nnf.replace('}', ']')
  str_nnf = str_nnf.replace(':', ',')
  str_nnf = str_nnf.replace("'CONCEPT', ", '')
  str_nnf = str_nnf.replace("'NOT', [", "['!', ")
  # str_nnf = str_nnf.replace("],", ",")
  str_nnf = str_nnf.replace("'OR', [", "['|', ")
  str_nnf = str_nnf.replace("'AND', [", "['&', ")
  str_nnf = str_nnf.replace("'EXISTS', [", "['*', ")
  str_nnf = str_nnf.replace("'FORALL', [", "['?', ")
  str_nnf = str_nnf.replace("'ROLE', ", '')
  
  str_nnf = '[' + str_nnf[1:].replace("[[", "['|', [")

  rep_str = re.search("\['[A-Za-z]+'\]", str_nnf)
  if rep_str:
    str_nnf = str_nnf.replace(str(rep_str.group()), "['!', " +str(rep_str.group())[1:])

  # print(str_nnf)
  Tbox.append(ast.literal_eval(str_nnf)[0])


for i in range(len(Tbox)):
  for j in range(len(Tbox[i])):
    # print(j)
    if type(Tbox[i][j]) == list:
      if Tbox[i][j][0] == '|' or Tbox[i][j][0] == '&':
        # print(len(Tbox[i][j]))

        if len(Tbox[i][j]) != 3:
          temp = Tbox[i][j]
          Tbox[i][j] = [Tbox[i][j][0], temp[:3], temp[3]]


# The ALC Tableau Algorithm

In [ ]:
def isConsistent(L):
  for a in L:
    for c in L[a]:
      if ['!',c] in L[a]:
        print('Inconsistancy: ',c,', ',['!',c],' in L(',a,'):',L[a])
        return False
  return True

In [ ]:
def tboxExhaust(L,R,Tbox):
  for C in Tbox:
    if isinstance(C,str) or C[0]=='!':
      for a in L:
        if C not in L[a]:
          return False
    else:
      if isinstance(C,list) and C[0]!='*':
        for a in L:
          if C not in L[a]:
            return False
  return True          



In [ ]:
def evaluate(La,Ra,Tboxa):
  L = copy(La)
  R = copy(Ra)
  Tbox = Tboxa.copy()
  print('L : ',L)
  print('R : ',R)
  print('Tbox : ',Tbox)
  
  if len(Tbox)==0:
    print('EMPTY TBOX!')
    return isConsistent(L)
  if not isConsistent(L):
    return False
  if tboxExhaust(L,R,Tbox) and isConsistent(L):
    print('TBOX COMPLETELY USED!')
    return True
  Tnow = Tbox[0]
  if isinstance(Tnow,list):
    if Tnow[0] == '&':
      print('AND RULE:',Tnow[0])
      Tnew = Tnow[1:].copy()+Tbox[1:]#+[Tbox[0]]
      return evaluate(L,R,Tnew)

    elif Tnow[0] == '|':
      print('OR RULE:',Tnow[0])
      Tnew1 = [Tnow[1]]+Tbox[1:].copy()#+[Tbox[0]]  
      Tnew2 = [Tnow[2]]+Tbox[1:].copy()#+[Tbox[0]]
      L1 = copy(L);L2 = copy(L);R1=copy(R);R2=copy(R)
      print('Split 1:')
      e1 = evaluate(L1,R1,Tnew1)
      print('Split 2:')
      e2 = evaluate(L2,R2,Tnew2)
      print('Split done: ',L,L1,L2)
      return e1 or e2

    elif Tnow[0] == '*':
      print('THERE EXISTS RULE:',Tnow[0])
      Lnew = copy(L)
      Rnew = copy(R)
      rel = Tnow[1]
      cl = Tnow[2]

      for a in L:
        Lnew[a].append(Tnow)
        for item in L:
          if not ((a+','+item) in R and rel in R[a+','+item] and cl in L[item]):
            Lnew['var'+a] = [cl]
            Rnew[a+','+'var'+a] = [rel]
      return evaluate(Lnew,Rnew,Tbox[1:].copy())#+[Tnow])    

    elif Tnow[0] == '?': 
      print('FOR ALL RULE:',Tnow[0])
      Lnew = copy(L)
      Rnew = copy(R)
      rel = Tnow[1]
      cl = Tnow[2]

      for a in Lnew:
        Lnew[a].append(Tnow)
        for item in L:
          if a+','+item in R and rel in R[a+','+item] and cl not in L[item]:
            Lnew[item].append(cl)
      return evaluate(Lnew,Rnew,Tbox[1:]+[Tbox[0]]) 

    elif Tnow[0] == '!':      
      for a in L:
        L[a].append(Tnow)
      return evaluate(L,R,Tbox[1:]+[Tnow])
      
    else:
      assert False, "WRONG SYNTAX"  
  else:
    for a in L:
      L[a].append(Tnow)
    return evaluate(L,R,Tbox[1:]+[Tnow])      

In [ ]:
def printEntailment(L,R,Tbox):
  e = evaluate(L,R,Tbox)
  if e:
    print('Model exists => Query not entailed by KB')
  else:
    print('Model does not exists => Query is entailed by KB')  

## Example 1

In [ ]:
printEntailment(L1,R1,Tbox)

L :  {'Priya': ['Mother'], 'Dhruv': ['Bachelor', ['?', 'parentOf', ['!', 'Person']]]}
R :  {'Priya,Dhruv': ['parentOf']}
Tbox :  [['|', 'Mammal', ['&', ['!', 'Person'], 'Biped']], ['&', ['|', ['!', 'Woman'], ['&', 'Person', 'Female']], ['|', ['|', ['!', 'Person'], ['!', 'Female']], 'Woman']], ['&', ['|', ['!', 'Man'], ['&', 'Person', 'Male']], ['|', ['|', ['!', 'Person'], ['!', 'Male']], 'Man']], ['&', ['|', ['!', 'Father'], ['&', 'Man', ['*', 'parentOf', 'Person']]], ['|', ['|', ['!', 'Man'], ['?', 'parentOf', ['!', 'Person']]], 'Father']], ['&', ['|', ['!', 'Mother'], ['&', 'Woman', ['*', 'parentOf', 'Person']]], ['|', ['|', ['!', 'Woman'], ['?', 'parentOf', ['!', 'Person']]], 'Mother']], ['&', ['|', ['!', 'Bachelor'], ['&', 'Man', ['?', 'husbandOf', ['!', 'Person']]]], ['|', ['|', ['!', 'Man'], ['*', 'parentOf', 'Person']], 'Bachelor']]]
OR RULE: |
Split 1:
L :  {'Priya': ['Mother'], 'Dhruv': ['Bachelor', ['?', 'parentOf', ['!', 'Person']]]}
R :  {'Priya,Dhruv': ['parentOf']}
Tbox :

## Example 2

In [ ]:
print('EXAMPLE 2')

EXAMPLE 2


In [ ]:
L = {'a':['T']}
R = {}
Tbox = ['C',['!','C']]

In [ ]:
print('Is KB consistent?:',evaluate(L,R,Tbox))

L :  {'a': ['T']}
R :  {}
Tbox :  ['C', ['!', 'C']]
L :  {'a': ['T', 'C']}
R :  {}
Tbox :  [['!', 'C'], 'C']
L :  {'a': ['T', 'C', ['!', 'C']]}
R :  {}
Tbox :  ['C', ['!', 'C']]
Inconsistancy:  C ,  ['!', 'C']  in L( a ): ['T', 'C', ['!', 'C']]
Is KB consistent?: False


## Example 3

In [ ]:
print('EXAMPLE 3')

EXAMPLE 3


In [ ]:
Rel = {'Likes':{'Lucy':['Apple']}}
Class = {'Fruit':['Apple'],'Person':['Lucy']}

In [ ]:
L ,R= abox2L(Rel,Class)
Tbox = []

In [ ]:
print(L)
print(R)

{'Apple': ['Fruit'], 'Lucy': ['Person']}
{'Lucy,Apple': ['Likes']}


In [ ]:
# Given that Lucy likes apple and no other information about
# her likes/dislikes, can we conclude that Lucy likes fruits?
Query = ['Lucy',['*','Likes','Fruit']]

In [ ]:
Lq,Rq = addQuery(Query,L,R)

In [ ]:
print(Lq)
print(Rq)

{'Apple': ['Fruit', ['!', 'Fruit']], 'Lucy': ['Person', ['?', 'Likes', ['!', 'Fruit']]]}
{'Lucy,Apple': ['Likes']}


In [ ]:
printEntailment(Lq,Rq,Tbox)

L :  {'Apple': ['Fruit', ['!', 'Fruit']], 'Lucy': ['Person', ['?', 'Likes', ['!', 'Fruit']]]}
R :  {'Lucy,Apple': ['Likes']}
Tbox :  []
EMPTY TBOX!
Inconsistancy:  Fruit ,  ['!', 'Fruit']  in L( Apple ): ['Fruit', ['!', 'Fruit']]
Model does not exists => Query is entailed by KB


## Example 4

In [ ]:
print('EXAMPLE 4')

EXAMPLE 4


In [ ]:
Rel = {'Likes':{'Lucy':['Apple']}}
Class = {'Fruit':['Apple'],'Person':['Lucy']}

In [ ]:
L ,R= abox2L(Rel,Class)
Tbox = []

In [ ]:
print('Before adding not of query')
print(L)
print(R)

Before adding not of query
{'Apple': ['Fruit'], 'Lucy': ['Person']}
{'Lucy,Apple': ['Likes']}


In [ ]:
# Given that Lucy likes apple and no other information about
# her likes/dislikes, can we conclude that Lucy likes ONLY fruits?

Query = ['Lucy',['?','Likes','Fruit']]

In [ ]:
Lq,Rq = addQuery(Query,L,R)

In [ ]:
print('After adding not of query')
print(Lq)
print(Rq)

After adding not of query
{'Apple': ['Fruit', ['*', 'Likes', ['!', 'Fruit']]], 'Lucy': ['Person', ['*', 'Likes', ['!', 'Fruit']]], 'varApple': [['!', 'Fruit']], 'varLucy': [['!', 'Fruit']]}
{'Lucy,Apple': ['Likes'], 'Apple,varApple': ['Likes'], 'Lucy,varLucy': ['Likes']}


In [ ]:
printEntailment(Lq,Rq,Tbox)

L :  {'Apple': ['Fruit', ['*', 'Likes', ['!', 'Fruit']]], 'Lucy': ['Person', ['*', 'Likes', ['!', 'Fruit']]], 'varApple': [['!', 'Fruit']], 'varLucy': [['!', 'Fruit']]}
R :  {'Lucy,Apple': ['Likes'], 'Apple,varApple': ['Likes'], 'Lucy,varLucy': ['Likes']}
Tbox :  []
EMPTY TBOX!
Model exists => Query not entailed by KB


## Example 5

In [ ]:
print('EXAMPLE 5')

EXAMPLE 5


In [ ]:
Rel = {'R':{}}
Class = {'T':['a']}

In [ ]:
# Does { T(a), C in D } entail (∃R•C in ∃R•D)?
L ,R= abox2L(Rel,Class)
Tbox = [['|',['!','C'],'D'],
        ['&',['*','R','C'],['?','R',['!','D']]]] #<= not of Query

In [ ]:
print('Just checking consistancy of system')
print(L)
print(R)

Just checking consistancy of system
{'a': ['T']}
{}


In [ ]:
for a in L:
  print(a)

a


In [ ]:
printEntailment(L,R,Tbox)

L :  {'a': ['T']}
R :  {}
Tbox :  [['|', ['!', 'C'], 'D'], ['&', ['*', 'R', 'C'], ['?', 'R', ['!', 'D']]]]
OR RULE: |
Split 1:
L :  {'a': ['T']}
R :  {}
Tbox :  [['!', 'C'], ['&', ['*', 'R', 'C'], ['?', 'R', ['!', 'D']]]]
L :  {'a': ['T', ['!', 'C']]}
R :  {}
Tbox :  [['&', ['*', 'R', 'C'], ['?', 'R', ['!', 'D']]], ['!', 'C']]
AND RULE: &
L :  {'a': ['T', ['!', 'C']]}
R :  {}
Tbox :  [['*', 'R', 'C'], ['?', 'R', ['!', 'D']], ['!', 'C']]
THERE EXISTS RULE: *
L :  {'a': ['T', ['!', 'C'], ['*', 'R', 'C']], 'vara': ['C']}
R :  {'a,vara': ['R']}
Tbox :  [['?', 'R', ['!', 'D']], ['!', 'C']]
FOR ALL RULE: ?
L :  {'a': ['T', ['!', 'C'], ['*', 'R', 'C'], ['?', 'R', ['!', 'D']]], 'vara': ['C', ['!', 'D'], ['?', 'R', ['!', 'D']]]}
R :  {'a,vara': ['R']}
Tbox :  [['!', 'C'], ['?', 'R', ['!', 'D']]]
L :  {'a': ['T', ['!', 'C'], ['*', 'R', 'C'], ['?', 'R', ['!', 'D']], ['!', 'C']], 'vara': ['C', ['!', 'D'], ['?', 'R', ['!', 'D']], ['!', 'C']]}
R :  {'a,vara': ['R']}
Tbox :  [['?', 'R', ['!', 'D']], 

## Example 6

In [ ]:
print('EXAMPLE 6')

EXAMPLE 6


In [ ]:
#  Jack is a landlord.
Rel = {'R':{}}
Class = {'L':['Jack']}
L ,R= abox2L(Rel,Class)

In [ ]:
print(L,R)

{'Jack': ['L']} {}


In [ ]:
# Can we conclude that Jack is a taxpayer?
Query = ['Jack','P']
Lq,Rq = addQuery(Query,L,R)

In [ ]:
print(Lq,Rq)

{'Jack': ['L', ['!', 'P']]} {}


In [ ]:
# Landlords own houses. Those who own taxable items are
# taxpayers. Houses are taxable.
Tbox = [['|',['!','H'],'B'],
        ['|',['!','L'],['*','O','H']],
        ['|','P',['?','O',['!','B']]],
        ]

In [ ]:
printEntailment(Lq,Rq,Tbox)

L :  {'Jack': ['L', ['!', 'P']]}
R :  {}
Tbox :  [['|', ['!', 'H'], 'B'], ['|', ['!', 'L'], ['*', 'O', 'H']], ['|', 'P', ['?', 'O', ['!', 'B']]]]
OR RULE: |
Split 1:
L :  {'Jack': ['L', ['!', 'P']]}
R :  {}
Tbox :  [['!', 'H'], ['|', ['!', 'L'], ['*', 'O', 'H']], ['|', 'P', ['?', 'O', ['!', 'B']]]]
L :  {'Jack': ['L', ['!', 'P'], ['!', 'H']]}
R :  {}
Tbox :  [['|', ['!', 'L'], ['*', 'O', 'H']], ['|', 'P', ['?', 'O', ['!', 'B']]], ['!', 'H']]
OR RULE: |
Split 1:
L :  {'Jack': ['L', ['!', 'P'], ['!', 'H']]}
R :  {}
Tbox :  [['!', 'L'], ['|', 'P', ['?', 'O', ['!', 'B']]], ['!', 'H']]
L :  {'Jack': ['L', ['!', 'P'], ['!', 'H'], ['!', 'L']]}
R :  {}
Tbox :  [['|', 'P', ['?', 'O', ['!', 'B']]], ['!', 'H'], ['!', 'L']]
Inconsistancy:  L ,  ['!', 'L']  in L( Jack ): ['L', ['!', 'P'], ['!', 'H'], ['!', 'L']]
Split 2:
L :  {'Jack': ['L', ['!', 'P'], ['!', 'H']]}
R :  {}
Tbox :  [['*', 'O', 'H'], ['|', 'P', ['?', 'O', ['!', 'B']]], ['!', 'H']]
THERE EXISTS RULE: *
L :  {'Jack': ['L', ['!', 'P'], 